# run.ipynb
Bu notebook, `torch_cnn` paketinin **çalıştığını** hızlıca doğrulamak ve `ReferenceNet` ile **tek forward / kısa train** denemesi yapmak için.

Repo kök dizininde açıp çalıştırman yeterli.


In [6]:
import os, sys, torch
print('python:', sys.version)
print('torch :', torch.__version__)
print('cuda  :', torch.cuda.is_available())
print('cwd   :', os.getcwd())


python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
torch : 2.5.1+cu121
cuda  : True
cwd   : c:\Users\hdgn5\OneDrive\Belgeler\GitHub\pytorch-cnn-architecture-library\Torch_CNN\torch_cnn


## Import testi
Eğer burada import patlıyorsa, repo kök dizininde değilsin veya `PYTHONPATH=.` ayarlamamışsındır.

In [7]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import torch_cnn
from torch_cnn.models import ReferenceNet

print("OK ✅", torch_cnn.__file__)


OK ✅ c:\Users\hdgn5\OneDrive\Belgeler\GitHub\pytorch-cnn-architecture-library\Torch_CNN\torch_cnn\__init__.py


In [8]:
from torch_cnn.models import ReferenceNet
print('ReferenceNet import: OK')


ReferenceNet import: OK


## 1) Hızlı forward (dummy input)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ReferenceNet(
    in_channels=3,
    num_classes=10,
    widths=(64, 128, 256),
    depths=(2, 2, 2),
    conv_kind='dynamic',
    norm='bn',
    act='relu',
    use_attention=True,
    attn_kwargs=dict(
        ca_reduction=16,
        ca_fusion='softmax',
        ca_gate='sigmoid',
        ca_temperature=0.9,
        coord_norm='bn',
        coord_dilation=2,
        residual=False,
        return_maps=False,
    ),
    dynamic_K=4,
    dynamic_reduction=4,
    dynamic_temperature=1.0,
).to(device)

x = torch.randn(8, 3, 32, 32, device=device)
with torch.no_grad():
    y = model(x)

print('x:', tuple(x.shape))
print('y:', tuple(y.shape))


x: (8, 3, 32, 32)
y: (8, 10)


## 2) CIFAR-10 Quick Train (opsiyonel)
Bu cell 1 epoch mini-train yapar. İstersen kapat geç.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim

tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

train_ds = datasets.CIFAR10(root='./data', train=True, download=True, transform=tf)
subset_idx = list(range(2048))
train_small = Subset(train_ds, subset_idx)

loader = DataLoader(train_small, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)

crit = nn.CrossEntropyLoss()
opt = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)

model.train()
loss_sum = 0.0
n = 0
for x, y in loader:
    x = x.to(device, non_blocking=True)
    y = y.to(device, non_blocking=True)
    opt.zero_grad(set_to_none=True)
    logits = model(x)
    loss = crit(logits, y)
    loss.backward()
    opt.step()
    loss_sum += float(loss.item()) * x.size(0)
    n += x.size(0)

print('mini-train loss:', loss_sum / max(n, 1))


## CLI ile çalıştırma
Notebook yerine script ile:

- Train: `cd Torch_CNN python -m torch_cnn.scripts.train --epochs 10 --conv dynamic --norm bn --act relu --attn`

- Eval : `python scripts/eval.py --ckpt results/checkpoints/reference_net_last.pt`


--------
---------
----------
--------
----------
----------
-----------
----------
----------
-------

In [12]:
import os, sys, time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

ROOT = os.path.abspath("..")  
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

import torch_cnn
from torch_cnn.models import ReferenceNet

print("OK  torch_cnn:", torch_cnn.__file__)
print("cwd:", os.getcwd())

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


from torchvision import datasets, transforms

train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])
test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
])

data_root = os.path.join(ROOT, "data")
train_ds = datasets.CIFAR10(root=data_root, train=True, download=True, transform=train_tf)
test_ds  = datasets.CIFAR10(root=data_root, train=False, download=True, transform=test_tf)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True,  num_workers=0, pin_memory=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False, num_workers=0, pin_memory=False)

model = ReferenceNet(
    in_channels=3,
    num_classes=10,
    widths=(64, 128, 256),
    depths=(2, 2, 2),
    conv_kind="standard",   
    norm="bn",
    act="relu",
    use_attention=True,
    attn_kwargs=dict(
        ca_reduction=16,
        ca_fusion="softmax",
        ca_gate="sigmoid",
        ca_temperature=0.9,
        coord_norm="bn",
        coord_dilation=2,
        residual=False,
        return_maps=False,
    ),
    dynamic_K=4,
    dynamic_reduction=4,
    dynamic_temperature=1.0,
).to(device)

crit = nn.CrossEntropyLoss()
opt = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)

@torch.no_grad()
def acc_top1(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

def train_one_epoch(epoch):
    model.train()
    total_loss, total_acc, n = 0.0, 0.0, 0
    t0 = time.time()

    for step, (x, y) in enumerate(train_loader, 1):
        x = x.to(device, non_blocking=False)
        y = y.to(device, non_blocking=False)

        opt.zero_grad(set_to_none=True)
        logits = model(x)
        loss = crit(logits, y)
        loss.backward()
        opt.step()

        bs = x.size(0)
        total_loss += float(loss.item()) * bs
        total_acc  += acc_top1(logits.detach(), y) * bs
        n += bs

        if step % 100 == 0:
            print(f"  [E{epoch:02d}] step {step:04d}/{len(train_loader)} | loss={loss.item():.4f}")

    dt = time.time() - t0
    return total_loss / max(n,1), total_acc / max(n,1), dt

@torch.no_grad()
def eval_one_epoch():
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0
    t0 = time.time()

    for x, y in test_loader:
        x = x.to(device, non_blocking=False)
        y = y.to(device, non_blocking=False)

        logits = model(x)
        loss = crit(logits, y)

        bs = x.size(0)
        total_loss += float(loss.item()) * bs
        total_acc  += acc_top1(logits, y) * bs
        n += bs

    dt = time.time() - t0
    return total_loss / max(n,1), total_acc / max(n,1), dt


EPOCHS = 10
t_all = time.time()

for ep in range(1, EPOCHS + 1):
    tr_loss, tr_acc, tr_t = train_one_epoch(ep)
    te_loss, te_acc, te_t = eval_one_epoch()

    print(
        f"Epoch {ep:02d}/{EPOCHS} | "
        f"train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} | "
        f"test_loss={te_loss:.4f} test_acc={te_acc:.4f} | "
        f"t_train={tr_t:.1f}s t_eval={te_t:.1f}s"
    )

print("done. total_time:", f"{time.time()-t_all:.1f}s")

OK  torch_cnn: c:\Users\hdgn5\OneDrive\Belgeler\GitHub\pytorch-cnn-architecture-library\Torch_CNN\torch_cnn\__init__.py
cwd: c:\Users\hdgn5\OneDrive\Belgeler\GitHub\pytorch-cnn-architecture-library\Torch_CNN\torch_cnn
device: cuda
Files already downloaded and verified
Files already downloaded and verified
  [E01] step 0100/391 | loss=1.4671
  [E01] step 0200/391 | loss=1.3281
  [E01] step 0300/391 | loss=1.1963
Epoch 01/10 | train_loss=1.3816 train_acc=0.4879 | test_loss=1.2634 test_acc=0.5524 | t_train=72.3s t_eval=5.6s
  [E02] step 0100/391 | loss=1.1581
  [E02] step 0200/391 | loss=0.9303
  [E02] step 0300/391 | loss=0.9941
Epoch 02/10 | train_loss=0.9293 train_acc=0.6676 | test_loss=0.9119 test_acc=0.6966 | t_train=72.1s t_eval=5.6s
  [E03] step 0100/391 | loss=0.6137
  [E03] step 0200/391 | loss=0.6484
  [E03] step 0300/391 | loss=0.8337
Epoch 03/10 | train_loss=0.7247 train_acc=0.7461 | test_loss=0.6884 test_acc=0.7668 | t_train=74.2s t_eval=5.7s
  [E04] step 0100/391 | loss=0.51